# VQE for H$_2$ molecule based on SNAP-displacement ansatz

## Prerequisite

Installation cells for Google Colab users.

In [1]:
!pip install qutip
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 10.3 MB/s eta 0:00:00


In [2]:
!pip install openfermion
!pip install openfermionpyscf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.3 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=981cc33f2af64db5195d133cd4e7aa9b6512b256fad1122e693fde5c23b7a6ad
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 51.3 MB/s eta 0:00:00


Import libaries.

In [3]:
import numpy as np
import qutip as qt
import scipy.optimize as sciopt

from scipy.linalg import expm
from functools import partial
from itertools import combinations

In [4]:
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.linalg import get_sparse_operator, jordan_wigner_sparse, expectation, get_ground_state
from openfermion.ops import FermionOperator
from openfermion.ops.representations import get_tensors_from_integrals
from openfermionpyscf import run_pyscf

In [5]:
import sys

In [6]:
import matplotlib.pyplot as plt

## Ansatz

### Basics

In [7]:
def np_arr_string(A):
    return np.array2string(A, separator=',')

In [8]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [9]:
def qproj00():
    return qt.basis(2, 0).proj()

def qproj11():
    return qt.basis(2, 1).proj()

def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)

def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)

def hadamard():
    op = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    return qt.Qobj(op)

In [10]:
def unpack_params_snap_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (nfock + 1)

    # Unpack
    alpha = X[:ndepth].copy()
    d1 = ndepth * nfock
    theta = X[ndepth:ndepth+d1].reshape((ndepth, nfock))

    return alpha, theta

def pack_params_snap_disp(alpha, theta):
    # Initialize
    ndepth = alpha.shape[0]
    nfock = theta.shape[1]
    dim = (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = alpha.copy()
    d1 = ndepth * nfock
    X[ndepth:ndepth+d1] = theta.reshape(-1)

    return X

### SNAP-displacement

Selective number-dependent arbitray phase (SNAP) and displacement operator ([reference](https://doi.org/10.1103/PhysRevA.92.040303))

\begin{align*}
U (\alpha, \vec{\theta})
&= S (\vec{\theta}) \: D (\alpha),
\\
S (\vec{\theta})
&= \sum_{n = 0}^{L - 1} \: \exp ( i \: \theta_n ) \: |n \rangle \langle n|,
\\
D (\alpha)
&= e^{ \alpha \: ( a^\dagger - a ) }.
\end{align*}

In [11]:
def snap_disp_op(alpha, thetavec):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # Initialize
    nfock = thetavec.shape[0]

    # SNAP
    S2 = np.exp(1j * thetavec[0]) * qt.basis(nfock, 0).proj()
    for i in range(1, nfock):
        S2 += np.exp(1j * thetavec[i]) * qt.basis(nfock, i).proj()

    # Rotation
    D2 = qt.displace(nfock, alpha)

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\alpha}, \bar{\theta})
= U (\alpha_{N_d}, \vec{\theta}_{N_d}) \cdots
U (\alpha_1, \vec{\theta}_1),
$$

where $\vec{\alpha}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L} $ is a matrix.


In [12]:
def snap_disp_ansatz(alphavec, thetamat, nfock):
    """
    SNAP-displacement ansatz.

    Arguments:
    alphavec -- displacement parameters
    thetamat -- SNAP parameters
    nfock -- Fock cutoff
    """
    # Check
    if thetamat.shape[0] != alphavec.shape[0]:
        raise ValueError("Lengths of theta and alpha do not match.")
    if thetamat.shape[1] != nfock:
        raise ValueError("Incorrect nfock chosen.")

    # Initialize
    ndepth = thetamat.shape[0]
    uni = snap_disp_op(alphavec[0], thetamat[0, :])

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = snap_disp_op(alphavec[i], thetamat[i, :])
        uni = ( new_uni * uni )

    return uni.full()

## Approximate Hamiltonian

\begin{align*}
H_Q
&= g_1
+ g_2 \: ( Z_0 + Z_1 )
+ g_3 \: ( Z_2 + Z_3 )
+ g_4 \: Z_0 Z_1
+ g_5 ( Z_0 Z_2 + Z_1 Z_3 )
+ g_6 ( Z_0 Z_3 + Z_1 Z_2 )
\\
&+ g_7 \: Z_2 Z_3
+ g_8 ( X_0 Y_1 Y_2 X_3
+ Y_0 X_1 X_2 Y_3
- X_0 X_1 Y_2 Y_3
- Y_0 Y_1 X_2 X_3 )
\end{align*}

### Imported data

Data for $N_d = 16$.

In [13]:
ZIII_nd16_loss = 8.338301805307009e-14

ZIII_nd16_alpha = np.array([1.60485302,2.217903  ,0.44674967,2.65327162,1.894671  ,2.21103061,
 0.48802542,2.04571748,1.70750064,2.10728167,0.98534594,2.12467184,
 1.05917446,1.44091935,1.19343878,2.79522351])

ZIII_nd16_theta = np.array([[ 2.98115175, 1.47853262, 0.66447648, 1.73919197, 1.85165904, 0.63776872,
   1.96100332, 1.25415948, 1.06216155, 0.24826121, 2.55346934, 1.64293168,
   3.74834504, 0.93724596, 3.43342755, 0.03954332],
 [ 1.97300588, 2.34197331, 1.31137496, 1.16766399, 2.38960616, 0.33352534,
   2.49020923, 1.8693111 , 2.08006331, 1.82653119, 2.86102621, 1.58749059,
   2.88138122, 2.44837729, 2.92769374, 0.85601546],
 [ 0.67085456, 2.02469974, 4.11965823, 3.69585694, 0.49427062, 2.73119819,
   1.34581619,-0.0340653 , 0.99919597, 2.66976694,-0.51399475, 1.15593401,
   3.22006143, 2.40201651, 2.36546283, 1.44318054],
 [ 1.7682015 , 1.10332704, 1.92823273, 1.13027146, 0.99602848, 0.49996281,
   2.3766625 , 0.98900669, 3.57368653, 2.73486715, 0.93555598, 1.11930634,
   3.23518916, 2.70438648, 1.7658476 , 1.51493156],
 [ 2.71084745, 2.8868929 , 2.0157058 , 1.13085759, 1.46505579, 2.7413559 ,
   0.46437017, 2.65716947, 0.74149859, 0.45179004, 2.09106972, 2.06431941,
   1.12255229, 2.82813766,-1.14362792, 1.87543013],
 [ 0.81004356, 0.164434  , 1.21064121, 2.43856282,-0.23225215, 3.96902162,
   3.24635032,-0.05986364, 2.17299476, 1.44510027, 0.94928636, 0.86441582,
   2.43022846,-0.22715583, 2.38516877, 0.65904183],
 [ 0.82673597,-0.07335323, 2.55364632, 0.38151327, 2.31798424, 3.06184886,
   3.02687954, 2.02877971, 2.06801887, 2.93558632, 1.05502887, 2.2969462 ,
   2.09184961, 0.11234192, 2.97226368, 0.92412248],
 [ 2.37250659, 1.46888245, 1.60400637, 2.90888138, 1.42295693, 0.31260967,
   1.56122342, 0.86050123, 0.84793142, 2.70637569, 1.53926098, 1.90911111,
   3.57484057,-0.01977969, 2.04419212, 2.60750218],
 [ 2.02146878, 1.06738038, 2.11439817, 3.26535207, 3.08269297, 1.63751174,
   3.57051137, 1.77014956, 0.06453272, 1.09162853, 0.23969402,-0.02559135,
   2.39412231, 2.03628764, 3.28572883, 1.25520718],
 [ 1.59736327, 0.30807608, 4.23264568, 3.48088442, 2.54162239, 2.07923948,
   3.59022595, 2.34859245, 1.07957244, 1.73286645, 3.12601268, 0.01188543,
   0.91289236, 0.67101907, 1.4240345 , 2.74009982],
 [ 2.39752814,-0.09031851, 1.07244239, 1.11450667, 0.18031355, 2.00397445,
   2.08950242, 0.66472919, 2.98667403, 1.83639409, 0.36356206,-0.07841126,
   1.510964  , 0.86930664, 3.90139724, 1.64879536],
 [ 3.23467961, 2.10766151,-0.24328052, 2.31829978,-0.5651385 ,-0.3565699 ,
   2.28865001, 2.89997702, 2.08350074, 0.50825763, 1.32445585, 1.68465105,
   4.58886294, 1.92807602, 0.76121197, 2.22168736],
 [ 1.8189073 , 0.40421868, 3.29598012, 1.65365839, 0.97051729, 2.75681918,
   1.06052716, 1.54278264, 0.7146392 , 0.4489492 , 3.03704492, 0.85107572,
   2.68436096, 0.56623291, 1.38685174, 1.221142  ],
 [ 1.94754956, 1.62835956, 0.13628077,-0.819753  , 1.85347438, 2.67571532,
   1.51233052, 3.30703599, 2.66027243, 2.17495659, 0.82725005, 0.98424832,
   2.1032482 , 0.50591393, 2.50821839, 4.46161654],
 [ 1.04502504, 0.3800717 , 0.32721728, 2.78788638, 3.00649763, 3.11073899,
   1.98089797, 0.57838666, 0.89867402, 0.95547977, 2.73938229, 0.52567818,
   3.25642389, 0.06292134, 0.04996013, 2.01245872],
 [ 0.9666247 , 2.34387434, 1.3501511 , 3.4623659 , 2.85226116, 0.99714606,
   1.41281632, 1.276999  , 1.67957921, 2.85313113, 0.29233122, 2.37789898,
   1.42148259, 1.75680895, 2.40807751, 0.11106967]])

In [14]:
IZII_nd16_loss = 1.076296105136616e-13

IZII_nd16_alpha = np.array([2.85721266,0.48691987,2.91022543,0.10710352,1.23225464,2.52862801,
 0.83420907,2.39767953,1.1290276 ,0.8905737 ,2.68096966,2.79834876,
 2.87661256,0.84791787,1.56014799,0.74290827])

IZII_nd16_theta = np.array([[ 1.04504722, 2.24792807, 2.47929825, 0.85834109, 0.30440268, 3.0371409 ,
   0.60664347, 2.75349802, 2.5391255 , 0.82680205, 1.12542017, 3.21125652,
   1.32250504, 2.58569758, 2.49545474, 0.7778232 ],
 [ 0.8623728 , 2.9551141 , 3.4670138 , 0.98235697, 1.5683556 , 3.33255622,
   1.7978664 , 1.56925612,-0.11954498, 2.53878724, 1.90149707, 2.71645386,
   2.02665469, 0.06422496, 1.46215822, 2.03620074],
 [ 0.60429731, 1.06296625, 1.49109971, 1.66883209, 1.71611771, 0.86614509,
   0.06596092, 1.99419469, 2.2685339 , 0.40961685, 2.44159957, 2.95244416,
   1.18303394, 2.71460333, 1.24947948,-1.4574756 ],
 [ 0.68226109, 3.50560823,-0.84185847, 1.80642028, 0.55676759, 2.90078419,
   0.3671949 , 2.02847901, 2.06450643, 0.33008177, 0.93895449,-0.45223867,
   0.50512815, 2.2212461 , 4.90091881, 4.64814818],
 [ 1.68676701, 0.57982884, 2.49860708, 1.71858782, 0.72323068, 0.46028485,
  -0.59011006, 3.16994033, 2.7896511 , 1.25856002, 1.99167583, 0.11811258,
   2.46899671, 0.32507936, 1.89476248, 1.00662724],
 [ 1.18873119, 1.01586193, 2.09055422, 1.51626089, 3.64296923, 2.48725976,
   2.68215239, 3.12131341, 2.69637296, 1.19721704, 2.2029387 , 0.52745372,
   0.35384105, 1.99210721, 1.91912196, 1.25716505],
 [-0.09296386, 2.42165594, 0.77702896, 0.45947823, 2.50537035, 0.66230157,
   3.94691528, 3.08118151, 2.95836616, 0.5893692 , 3.19286365, 0.33242303,
   3.34754096,-0.66630513, 0.53676447, 1.19945486],
 [ 1.61075305, 1.46622209, 1.98405202, 1.5956283 , 3.33446224, 1.66410131,
   3.67103125, 1.25562554, 2.72999229, 0.87574462, 1.69124971, 2.72043275,
   2.1065304 , 1.50857629, 2.24252822, 1.54331843],
 [ 1.38329693, 2.72999983, 2.65289859, 3.64983199, 1.32127479, 0.19464335,
   2.66638749, 0.76497507, 2.95010741, 3.35936912, 0.42265682, 0.06200985,
   1.81840701, 1.08434534, 2.11617088, 0.77649838],
 [ 2.78112083, 1.63845391, 2.27222057, 2.13467397, 1.34997908, 2.60577628,
   0.58096704, 1.75501152, 0.89599743, 0.84515862, 1.81283869, 2.69501254,
   1.79759301, 2.55655265,-0.29145442, 1.46484604],
 [ 0.42820369,-0.08419466, 1.94309528, 1.35386615, 1.88553005, 3.01268664,
   2.55348365, 0.12863759, 0.18093611, 2.14445334, 1.54355935, 0.18314602,
  -0.24330178, 1.08959203, 0.60668107, 2.58228511],
 [ 0.1754027 , 1.08070537, 2.35266807, 0.1872294 , 1.57122565, 3.33340392,
   1.0044761 , 0.51056233, 2.82016656,-0.0426178 , 1.83695935,-0.01842874,
  -0.07007198, 0.15335876, 1.71183212, 1.05467971],
 [ 0.7523043 , 1.88996699, 0.37450035, 1.59568639, 1.41697906, 2.84347371,
   2.60835172, 1.35814441,-0.09738587, 0.67967231, 0.3273583 , 0.71164252,
   1.51429288, 0.8107764 , 2.07066341, 2.21178155],
 [ 2.37615531, 2.05722195, 0.01181944, 2.12254411, 2.60016367, 0.48185586,
   2.11312026, 0.38049829, 0.34900231, 1.89613959, 1.7658842 , 1.93959214,
   0.65852103, 0.75722218, 3.46271411, 1.83852129],
 [ 2.40159756, 0.2470828 , 0.0654492 , 2.18523224, 1.7383153 , 1.65651857,
   0.08761015, 1.22781521, 2.63872767, 1.79304181,-0.03147106, 2.3507555 ,
   2.79251535, 2.26786009, 1.54748318, 0.1142182 ],
 [ 2.11556698, 0.29280627, 1.84961674, 0.28490582, 2.19598437, 3.78319866,
   1.69947899, 3.23398977, 0.54335294, 2.43666693, 1.42072694, 1.765961  ,
   1.43696085, 2.49362946, 1.40676859, 0.36930148]])

In [15]:
IIZI_nd16_loss = 3.386062556274457e-14

IIZI_nd16_alpha = np.array([0.96473929,0.78621302,2.52234209,0.91491321,2.57620561,2.54826898,
 1.77884147,0.66960455,1.26864394,1.83307156,0.30336203,2.79808138,
 1.95691417,1.7153176 ,2.90054323,1.78767558])

IIZI_nd16_theta = np.array([[ 2.90147113e+00, 7.77911239e-01, 2.05596176e+00, 3.08014649e+00,
   1.48052488e+00, 1.60101933e+00, 3.60890067e+00, 2.14711635e+00,
   1.35051246e+00, 8.43964632e-02, 1.36901545e+00, 8.99427587e-01,
   9.03165286e-01,-3.35804249e-01, 2.80602986e+00, 3.54722540e-01],
 [ 1.25313006e+00, 1.78716083e+00, 1.70616441e+00, 3.59545901e+00,
   3.22372816e+00, 5.23352834e-01, 2.72871497e+00, 2.02010918e+00,
   1.79437158e+00, 2.00159585e+00, 3.09704674e+00, 1.81667397e+00,
   2.47276219e+00,-2.18884418e-01, 1.98754100e+00, 8.33506903e-01],
 [-2.29351064e-01, 5.31473485e-01, 5.45065672e-01,-2.98362475e-03,
   2.44246717e+00, 2.63962481e+00, 2.96120016e+00,-3.52221012e-01,
   2.91871254e+00, 2.18949613e+00, 1.68106128e+00, 2.82893264e+00,
   4.65466213e-03, 3.07777173e+00, 2.36206803e+00, 2.05622450e+00],
 [ 2.10574443e+00, 1.84072198e+00,-4.83871672e-01,-8.80729910e-01,
   1.61362438e+00, 1.05045326e+00, 2.31651267e+00, 3.13889321e+00,
   1.12142638e-01, 1.84361514e+00, 2.33334116e+00, 1.96837069e+00,
   2.44492892e+00, 1.86928114e+00, 2.07359342e+00, 9.47180736e-01],
 [ 2.00977128e+00,-1.21040184e+00, 1.00516382e-01, 5.65208299e-01,
   2.21169144e+00, 5.07791401e-02, 1.48984923e+00, 1.25112659e+00,
   3.54124812e-01, 1.74191786e+00,-3.38222234e-01, 2.63173459e+00,
   1.51342947e+00, 2.15416075e+00, 1.20696247e+00, 9.67188929e-01],
 [ 4.09145616e-01, 1.76487124e+00, 8.62605904e-01, 1.97226421e+00,
   1.87772419e+00, 2.52031861e+00, 1.08969541e+00,-1.68353620e-01,
   1.18032232e+00, 1.98984842e+00, 2.69651787e+00, 1.39359065e+00,
   1.44327644e+00, 3.94002349e+00, 2.01561917e+00, 2.11378945e+00],
 [ 7.54568696e-01, 4.04860676e-01, 2.56960949e+00, 1.24990817e+00,
   2.19570499e+00, 8.78542666e-01, 5.09292067e-01,-1.26705808e-01,
   2.93027767e+00, 1.53002632e+00, 1.68641973e+00, 1.32543278e+00,
   1.96306318e+00, 3.19504029e+00, 1.31849256e+00, 1.38004492e+00],
 [ 3.08480003e+00, 4.17129419e+00, 1.37440914e+00,-4.09333834e-02,
   7.07746565e-01, 8.64991124e-01, 2.40156631e+00, 3.43136255e+00,
   2.37432995e-01, 2.74630050e+00,-5.99957981e-01, 1.31970475e+00,
  -5.12201466e-01, 2.05722607e+00, 4.82238042e-01, 2.25653813e+00],
 [ 1.03114531e+00, 9.78709188e-01, 1.77425328e+00, 3.50723626e+00,
   1.62206741e+00, 2.33888203e+00, 1.88254634e-01, 1.28815637e+00,
   1.37347953e+00, 1.12224676e-01, 2.69109265e+00, 3.19605298e-01,
   6.75453344e-01, 4.98829921e-01, 1.78167881e+00, 4.26084410e-01],
 [ 7.74038767e-01, 1.51777027e+00,-3.14210571e-02, 1.40071329e-01,
  -1.18764204e+00, 2.37433064e+00, 1.44124846e+00, 1.21187732e+00,
   3.42235069e+00, 3.65661600e+00, 1.67067270e+00, 7.64127621e-01,
   2.95945660e+00, 1.54691763e+00, 2.29933643e+00, 4.02729923e+00],
 [ 2.69473945e+00, 2.15455737e+00, 2.01588020e+00, 4.17518519e+00,
   2.57524832e+00, 5.36933539e-01, 2.66099856e+00, 9.78108339e-01,
   6.57642097e-01, 2.34972766e+00, 9.24079025e-02,-1.34181063e+00,
   2.13466756e+00,-3.64577007e-01,-1.20301754e+00,-3.96464040e-01],
 [ 1.19754143e+00, 4.89568314e-01, 1.70083480e+00,-3.83334333e-01,
   2.37667324e-01, 3.01970822e+00, 2.49822857e+00, 2.17997000e-01,
  -9.08899037e-01, 3.02651021e+00, 1.49981134e+00, 1.60595529e+00,
   2.03729891e+00, 1.69365193e+00, 2.06326327e+00, 9.34427282e-02],
 [ 2.32843715e+00, 1.05904376e+00, 2.58497310e+00, 1.56514061e+00,
   2.20823799e+00, 2.66713035e+00, 8.92480111e-01,-2.68468131e-01,
   5.82420982e-01, 1.25150486e+00, 1.65846460e+00, 2.77978028e+00,
   8.46620040e-01, 1.75773432e+00, 1.08686677e+00, 1.69108789e+00],
 [ 1.84810603e+00, 1.67299557e+00, 2.27414847e+00, 2.89927398e+00,
  -6.17925793e-01, 3.50174368e+00, 1.32544254e+00, 6.14008360e-01,
   1.84387875e+00, 1.59211161e+00,-3.15399190e-01, 1.71117309e+00,
   9.29000477e-01, 3.19296999e+00, 6.91413636e-01, 2.89252109e+00],
 [ 1.80341381e+00, 2.61397081e+00, 3.05528502e+00, 2.18764521e+00,
   1.23080683e+00, 1.34857561e+00, 1.32632220e+00,-8.99752811e-02,
   2.33555916e+00, 8.00504958e-01, 8.43138122e-01, 7.16515256e-01,
   1.39148068e+00, 2.36384895e+00, 2.88648493e+00, 2.25364541e+00],
 [ 1.97999504e+00,-6.60242166e-01, 4.76587760e-01, 5.93931729e-01,
   2.15604968e+00, 2.16659663e+00, 1.39523334e+00, 2.54258314e+00,
   2.94258370e+00, 2.80419239e+00, 3.06680604e-01, 2.38191235e+00,
   2.64017828e+00, 6.17025912e-01, 2.65488758e+00, 2.53833540e+00]])

In [16]:
IIIZ_nd16_loss = 8.81612803514647e-14

IIIZ_nd16_alpha = np.array([ 1.30141567, 2.12479533, 0.86254201,-0.21433731, 2.26289773, 2.7992863 ,
  0.76655434, 3.136044  , 1.38383788, 2.63491879, 2.85572283, 2.38866933,
  2.62413518, 2.21433678, 1.04007631, 1.45394358])

IIIZ_nd16_theta = np.array([[ 2.76587464, 0.76432195, 1.97916993, 0.73073089, 1.50770047, 0.05672322,
   2.09002191, 1.10483611, 0.47615719, 2.84370998,-0.42562786, 0.26720481,
   2.49206204, 1.32865321, 0.97245726, 1.54420116],
 [ 1.77649792, 2.09738889, 2.0713541 , 0.20949594,-0.87999637, 0.20509621,
   0.68485834, 1.18988355, 0.22758778, 1.02947395, 1.271127  , 2.08138718,
   0.20560574, 4.0052517 , 0.49309405,-0.39698553],
 [ 2.59731394, 0.2282598 , 2.15392821, 2.53162222, 2.77062996, 2.25324406,
   1.58482058, 2.14066002, 0.26749118, 0.97748788, 2.0405794 , 1.76976897,
   0.11241559, 2.15448335, 1.78859979, 0.15509694],
 [ 0.19127136, 3.6243453 , 1.29931837, 2.95148134, 3.4495508 , 0.31501603,
   0.65587841, 2.38077159, 1.27264409, 1.12187888, 0.61047423,-0.38685022,
   1.07688756, 1.72636526,-0.44582714, 2.21896576],
 [-0.58097731, 0.72147113, 0.1030444 , 2.67341225, 2.39807187, 3.34445512,
   3.36497988, 2.33273185, 2.20407895,-0.54570127, 1.72639564, 1.81858548,
   0.2530774 , 3.57642074, 0.13565304, 1.55551687],
 [ 2.71497389, 0.47103247, 1.71440422, 2.28013113, 2.60072698, 1.43678788,
   0.91985718,-0.0359042 , 1.8865416 ,-0.52008566, 0.95598323, 0.73279891,
   0.43079303, 0.76243785, 2.40336528, 3.48287691],
 [ 3.90666488, 2.47181395,-0.23303827, 2.60241012, 0.03398005, 1.80831628,
   1.73727065, 3.23618866, 1.21111096, 3.96331784, 2.20787336, 1.66943651,
  -1.18672752, 2.68871839, 1.31201891, 2.12133129],
 [ 0.95531458, 1.79872648,-0.16517167, 2.67970602, 0.18042328, 2.9326412 ,
   2.56266602, 1.53150092, 2.41698089, 2.27879925, 2.56794182, 1.06776648,
   0.09773524, 0.91957029, 2.32749748, 0.5129747 ],
 [ 0.26874319, 3.07100904, 1.26972465, 0.1458639 , 1.11640488, 0.91107889,
  -0.59797782, 3.26316462, 1.10015679, 0.46152167, 2.68841448, 1.9093647 ,
   1.37830808, 3.76050458, 2.93662058,-0.79376864],
 [ 0.81177919, 2.84098023, 2.6605459 , 0.73312671, 0.59309644, 2.54022234,
   0.74139782, 1.59968179,-0.05515084, 0.19082968, 1.81523255, 0.79959807,
   1.3743474 , 1.03038608, 2.76585447, 3.1834208 ],
 [ 2.16172645, 1.27651475, 2.92031815, 1.92942166, 0.38011015, 0.28069716,
   2.08516145, 2.73084846, 0.53355317, 0.70768127, 2.26885016, 1.91936507,
   3.12815057, 2.47208744, 1.0061701 , 0.72754546],
 [ 2.44332222, 2.63447207, 2.68411813, 0.717295  ,-1.04771528, 0.59096785,
   1.96636905, 0.79892096, 0.75266169, 2.62615722, 2.66596994, 0.31310479,
   2.8186962 , 3.26243441, 0.62179806, 1.98712888],
 [ 1.96261783, 2.52765567, 3.33731151, 2.45085012, 0.53945415, 2.24278918,
   1.6939284 , 3.47191323, 1.40876828, 1.0896059 , 1.1379982 ,-0.62533705,
   1.20665161, 1.90576247, 2.72783014, 0.26460507],
 [-0.17349645, 1.11591726,-0.02740046, 0.01296456, 2.3082642 , 3.0179977 ,
   1.51621057, 0.97532791, 0.40496243, 2.6796001 , 1.02872912, 1.90324812,
   1.32949312, 0.62089565, 1.26456329,-0.11369871],
 [ 3.1616339 , 1.36701477, 3.30136812, 2.50751854, 2.15367406, 1.53260023,
   2.05733653, 2.76719761, 1.33475034, 2.99534691, 0.79375719, 0.45562247,
   2.37882614, 0.83195918, 0.51362942, 0.52973703],
 [ 1.70975516,-0.75203565, 0.3712709 , 1.40517175, 1.07622027, 3.60513588,
  -0.82361391, 2.09052827, 1.89126811, 1.45324698, 1.94751095, 3.07022686,
   3.58221066, 0.58296289, 2.13898222, 1.2675353 ]])

In [17]:
ZZII_nd16_loss = 7.729147832417361e-15

ZZII_nd16_alpha = np.array([0.38481054,1.67203606,1.08859491,1.73245734,0.96244684,2.1549959 ,
 1.17328034,1.66003939,0.76342237,1.20384802,2.34822652,0.44396039,
 1.22650598,0.67878601,2.76833228,1.35636813])

ZZII_nd16_theta = np.array([[ 2.66631073, 2.41143572, 0.62903781, 3.24990986, 1.27231929, 3.3678526 ,
   0.43085293, 2.63521283, 2.62868472, 1.43724478, 0.94419304, 1.18858918,
   0.83455681,-0.53368687, 3.52246067, 1.23671011],
 [ 1.90456053, 1.98557651, 0.37428561, 2.58512532, 2.58012245,-0.02081942,
   2.74935435, 0.94222234, 3.17713286, 2.52089457, 2.69633716,-0.27593502,
  -0.25827304, 1.75606453, 2.50150016,-0.03572691],
 [ 0.43629759, 3.22781471, 2.83057764, 2.45685795, 2.04855259, 2.52041432,
  -0.21162064, 2.66419767, 1.84495862, 2.52265786, 3.33161128, 1.70785258,
   1.79739067, 2.74052698, 2.84215533, 2.3082544 ],
 [ 1.76539766, 0.82037489, 0.03141619, 2.06556981, 1.91728295, 2.40143815,
   0.76445792, 2.79466595, 1.50271588, 2.99934175, 0.86048549, 1.09542094,
   1.24249247, 2.08859376, 2.53627332, 0.93679035],
 [ 2.47928734,-0.1345825 , 2.30747023, 3.34782314, 0.82100421, 0.94971119,
   1.58873309, 0.61136052, 0.92520379, 0.25705501, 1.63979509, 0.9835475 ,
   0.02317042, 0.62472192, 0.77608644, 0.53506341],
 [-0.40021054, 3.88940946, 3.03669227, 2.88453173,-1.32164564, 0.71771056,
   4.11083528, 2.05418106, 2.84368917, 2.77926462, 0.99646867,-0.18610718,
   0.86124408, 0.35115582, 0.63901211, 4.55085561],
 [ 2.34199926, 1.63032837, 3.34849033, 1.82712422, 1.31769775, 0.65866272,
   1.97735137, 1.65375613, 1.02191301, 1.14167114, 2.61543177, 3.01403155,
   0.04786797, 1.64460015, 0.85799196, 1.55760238],
 [ 0.03251075,-0.62698777, 2.81203871, 2.36730817, 2.70912102, 1.39408407,
   1.74145754, 2.61670814, 2.2634593 , 1.14589647, 1.29123599, 2.02409024,
   2.56989333, 0.63620245, 3.06643084, 3.503516  ],
 [ 1.00781442, 1.31492799, 0.85293036, 2.66255188, 2.66467979, 2.98522767,
  -0.11093901,-0.10972785, 1.36013695, 0.89387192, 2.79078771, 1.88725494,
   0.19116517, 0.58805688, 2.7983271 , 0.94408321],
 [ 1.42214989, 3.21268303, 2.25037414, 2.32344658, 2.08218664, 1.58722761,
   0.78669632,-0.28956543, 0.32757851, 0.6956433 , 0.38523534, 0.94616492,
   2.23053128, 0.68105918, 2.10321629, 2.17018443],
 [ 2.32459584, 1.64221881, 2.06825009, 1.06868454, 1.68456039, 0.49326195,
  -0.29269706, 3.45727376, 2.54816972, 2.73256727, 2.26438987, 2.15721069,
   2.8792439 , 0.63680343, 1.8786951 , 0.392378  ],
 [ 1.41386488, 2.28644826, 0.14344155, 3.1303836 , 2.95470595, 0.40735085,
   1.43004631, 2.82627564, 3.05196726, 0.71262145, 1.46823246, 2.25152712,
   0.7769002 , 3.36917993, 2.08238893, 0.99819154],
 [ 3.29802697, 2.72381876, 1.77894575, 0.47108385, 2.39638474,-0.14798832,
   1.64059917, 0.73106547, 0.70455941, 2.81721168, 0.45758559, 2.96408778,
   0.5529808 , 0.104461  , 0.42419247, 1.64399088],
 [-0.3649349 , 0.44847027, 3.3881977 , 0.40240797, 0.74106187, 2.81704128,
   1.85752012, 0.98582657, 3.06784249, 0.8285832 , 2.07624305, 2.17963971,
   2.58736172, 1.41688236, 1.39828678, 0.08673085],
 [ 1.77430985, 1.68094597, 1.50659213, 0.31851752, 1.49994661, 1.45317932,
   2.84147856, 0.15841837, 1.64712205, 2.89881296, 0.71633536, 0.57880171,
   3.56213671,-0.17440251, 3.18356635, 1.16925266],
 [ 3.53495488, 1.8465058 , 2.02430197, 2.28910934, 3.03267701, 2.09434612,
  -0.15261725, 1.5366506 , 1.66064063, 1.41092111, 1.28464899, 1.76854385,
   1.65016621, 1.85858468, 3.14638566, 2.07797529]])

In [18]:
ZIZI_nd16_loss = 3.026387872867353e-14

ZIZI_nd16_alpha = np.array([2.52105202,1.35091325,0.60982996,1.69971429,0.10052046,2.33990878,
 1.53801746,1.73235922,0.83214971,2.62994314,0.84179596,1.13562956,
 3.14192756,1.17701481,2.65832091,2.31828646])

ZIZI_nd16_theta = np.array([[ 4.16254249e-01, 4.46960054e+00, 3.18044758e+00, 2.56829750e+00,
   8.76371578e-01, 8.89305765e-02, 2.85096165e+00, 1.00350242e+00,
   9.36791346e-01,-2.54228495e-01, 2.02847175e+00, 1.05249288e+00,
   3.01613416e+00, 8.47813554e-01, 2.40988976e+00, 2.58317575e+00],
 [ 7.90712893e-01, 3.20120109e+00, 2.20149397e+00, 3.91119916e+00,
   9.09494523e-01,-1.48665401e-01, 1.21241655e+00, 2.46909601e+00,
   1.56384953e+00,-3.35230838e-02, 1.37055843e+00, 1.02679793e+00,
   2.16865708e+00, 6.83459723e-01, 3.27842088e+00, 2.86040563e+00],
 [ 4.80949218e-01, 2.31793491e+00, 1.08723073e+00, 5.49844483e-01,
   1.54540299e+00, 2.22198589e+00, 3.82030862e+00, 1.14086684e+00,
   2.60741558e+00, 5.83726603e-01, 2.61126205e+00, 5.39497702e-01,
   1.50815896e-01, 1.53985896e+00, 6.32652693e-01, 1.41824662e-01],
 [ 1.23396510e+00, 1.76908461e+00, 3.75146638e-01, 4.11880647e+00,
   1.54880379e+00,-1.27167550e+00, 3.18969189e+00, 1.54013887e+00,
   2.41748625e+00, 2.77712987e+00, 3.06307831e+00, 7.72851396e-01,
   5.63991615e-01, 3.31931364e+00,-4.24326281e-02, 5.10782441e-01],
 [ 4.12477128e-01, 2.59501618e+00,-2.91204070e-01, 1.82689735e-01,
   9.93195204e-01, 3.89918642e+00,-5.87713754e-01, 1.34087385e+00,
   2.43758704e+00, 2.47198882e+00, 1.79053839e+00, 1.95826854e+00,
   3.18321700e+00, 3.26637967e+00, 3.75249191e+00, 8.84760783e-01],
 [ 1.97544526e+00, 7.45868648e-01, 1.20745623e+00, 1.00447972e+00,
   3.34150379e+00, 8.97033043e-01, 1.79675402e+00, 8.87594364e-02,
   2.86912262e-01, 1.62133418e+00, 7.45712414e-01, 2.68998066e+00,
   1.80998035e+00, 1.07318799e+00, 2.59033024e+00, 2.77301317e-01],
 [-1.68133810e+00, 3.10354308e-01, 1.04773169e+00, 2.97176744e+00,
   2.89995876e+00, 2.98295122e+00, 2.90081031e+00, 1.95177656e-01,
   3.70372976e+00, 3.46394306e-01, 2.73194777e+00, 3.22770518e+00,
   3.29979361e+00, 8.00697315e-01,-5.09128825e-01, 1.93086924e+00],
 [ 2.08901444e+00, 2.76058445e+00, 4.81434002e-01, 1.19974146e+00,
   3.17582427e+00, 7.45690582e-01, 2.25204638e+00, 8.11585814e-02,
   2.49446478e+00, 2.08493747e+00, 1.96601482e+00, 1.43856285e+00,
   9.35944542e-01, 4.53542516e-01, 9.14734835e-01, 1.81334384e+00],
 [ 1.41391168e-01, 2.77558880e+00, 2.43755205e+00, 4.96791373e-01,
   5.72969162e-01, 7.97243848e-01, 3.78167212e-01, 1.99958754e+00,
   1.28756295e+00, 1.94815536e+00, 2.90746984e+00, 2.54655789e+00,
   1.90597418e+00, 1.03690069e+00, 1.36828853e+00,-7.69502751e-01],
 [-6.33236628e-01, 3.90116759e+00, 1.57972595e+00, 1.43965474e+00,
   2.26141974e+00, 7.15528660e-01, 2.56143675e+00, 7.44191274e-01,
   9.54467442e-01, 2.82955701e+00,-4.39335290e-01, 1.33811068e+00,
   3.61302110e+00,-4.56707827e-01, 7.24875883e-01, 1.13522448e+00],
 [ 3.20893666e+00, 2.72789482e+00, 1.75539267e+00, 2.37041974e-01,
   6.64905775e-01, 3.46140128e+00, 4.90054573e-01, 3.19538869e-01,
   2.46992034e+00, 2.03980555e+00, 1.97001476e-01, 2.69234423e+00,
   2.46430134e+00,-3.22409381e-01, 2.22205525e+00, 3.13976041e-01],
 [ 8.58358365e-01, 1.99979288e+00, 1.03698432e+00, 1.62987074e+00,
   2.00553037e+00, 1.64036308e+00, 1.76914684e+00,-8.87830584e-02,
   3.71219743e+00,-9.36475736e-01, 1.77288661e+00,-5.43732635e-01,
   4.36838830e+00, 1.39846539e+00, 3.34666709e-01, 1.63586305e+00],
 [ 2.17164905e+00, 1.35461092e+00, 6.46181640e-01, 9.74023597e-01,
   2.54132782e+00,-1.72951889e+00, 2.85204987e+00, 7.53419927e-01,
   7.80028623e-01, 6.22882569e-01, 1.82921155e+00, 3.24291521e+00,
   5.89223183e-01, 2.26666639e+00, 1.38954244e+00, 7.80440381e-02],
 [ 1.48669268e+00, 5.30136038e-01, 2.43503608e+00, 1.90565447e+00,
   3.30488577e+00, 3.12474488e+00, 2.04634911e+00, 9.93890932e-01,
   1.99970675e+00, 2.34936572e+00, 2.31648154e+00, 1.69044487e+00,
  -4.29768617e-04, 1.43425409e+00, 3.29200756e+00, 2.70746395e+00],
 [ 1.20890521e+00, 6.68859647e-01, 2.10924224e+00, 1.86837203e+00,
   2.91791657e+00, 4.48175149e+00, 1.80194005e+00, 1.27773271e-01,
   1.82876581e+00, 1.42170792e+00, 3.64559601e+00, 1.73878505e+00,
   1.48595983e+00, 1.31972714e+00, 1.34416906e+00, 1.70326810e-01],
 [ 1.18091940e+00, 8.31153278e-01, 1.58528815e+00, 2.38663363e+00,
   4.79018771e-01, 9.63307070e-01, 1.10001824e+00, 8.21470044e-01,
   5.80424576e-01, 2.86985149e+00, 3.73239767e+00,-1.10922254e+00,
   2.33329208e+00, 3.85366602e+00, 2.67514137e+00, 1.99799788e-01]])

In [19]:
ZIIZ_nd16_loss = 3.561113325846533e-14

ZIIZ_nd16_alpha = np.array([0.77294065,0.2035196 ,1.50643314,0.26320996,1.28796145,0.47652586,
 1.52624093,1.28250209,1.29115063,3.01610176,2.4342019 ,2.10624783,
 1.96492247,0.53618314,2.29869642,1.12810029])

ZIIZ_nd16_theta = np.array([[-4.79284681e-01, 3.50585344e+00, 2.35014707e+00, 9.39074040e-01,
   1.81305255e+00, 2.66462047e+00, 1.67381979e+00, 2.05019578e+00,
   1.22127113e+00,-1.14770196e-01, 1.35596431e+00, 1.71429463e+00,
   5.01995391e+00, 1.92068716e+00, 8.50566165e-01, 2.23102556e+00],
 [ 3.07119249e+00,-7.43809811e-01, 1.18420382e+00, 1.73445547e+00,
   7.26987033e-01, 1.21855666e+00, 8.37459316e-01, 3.38684805e+00,
   2.54065055e+00,-1.61914074e-01, 2.29036074e-01, 4.58258563e-01,
   2.81667034e+00, 2.97957231e+00,-8.99363045e-02, 9.08074952e-01],
 [-1.74906831e-01, 5.36607570e-01, 4.65276764e-01,-2.17411134e-01,
   2.55612216e+00, 2.63476822e+00, 2.66537960e+00, 1.80881040e+00,
   2.21610565e+00, 8.07906368e-01,-2.71252640e-01, 3.00095352e+00,
   6.98959680e-01, 2.66000452e+00, 7.34280519e-02, 1.48024863e+00],
 [ 3.02515933e+00, 2.84265305e+00, 2.14874331e+00, 9.90209035e-01,
   4.81584059e-01, 1.18732906e+00, 2.18222744e+00, 1.80575118e+00,
   8.65860152e-01, 1.59487766e+00, 1.84671620e+00, 3.54009482e+00,
   5.74747147e-01, 2.56448706e-01, 1.61088739e+00, 1.07610640e+00],
 [ 1.94225184e+00,-4.34260994e-01,-9.54525547e-01, 2.24112726e+00,
   2.75508439e+00, 2.44746130e+00, 1.55302863e+00, 2.24476271e+00,
   4.73485482e-03, 2.66028589e+00, 2.89054561e+00, 2.52848821e+00,
   4.56376870e-01,-8.50888102e-01,-2.02713373e-01, 3.36381599e+00],
 [ 2.75168812e+00, 1.89093258e+00, 3.26234586e+00,-1.63023708e+00,
   1.18381388e+00, 2.01397614e-02, 2.88825697e+00,-7.00464663e-01,
   2.07639696e+00, 3.74902828e+00, 3.37389038e+00, 2.37584353e+00,
   5.15706768e-01, 2.14567262e+00, 3.57939672e+00, 2.17182537e+00],
 [ 2.48756511e-01, 3.17916594e+00, 2.76940766e+00, 3.33950789e+00,
   1.73817082e+00, 2.42640462e+00, 1.23150414e+00, 1.02919583e-01,
  -1.08313487e-01,-2.35289939e-01,-1.50270873e-01, 3.65780008e+00,
   1.41838808e+00,-3.87932175e-01, 2.17078416e+00, 2.48404446e+00],
 [ 2.73906379e+00, 2.56866209e+00, 3.38713515e+00, 3.01498282e+00,
   2.62547462e+00, 2.07067272e+00, 1.65700992e+00, 4.34361975e-01,
   1.61312327e+00, 2.72168325e+00, 1.38230205e+00, 4.40190335e-01,
  -4.23513340e-01, 2.34428327e+00, 1.51975015e+00, 2.11240901e+00],
 [-1.62056172e+00, 1.36978305e+00, 2.55968044e+00,-2.41039495e-01,
   2.06036010e+00,-3.37901113e-01, 1.22784480e+00, 9.21105725e-01,
   2.74555430e+00, 1.16732019e+00, 2.05824941e+00, 1.91147304e+00,
   1.82611277e+00, 2.48967132e+00, 1.63560292e+00, 3.94214173e+00],
 [-1.86137106e+00, 2.82840752e+00, 6.97032849e-01, 1.33416005e+00,
   1.55530657e+00, 1.54167449e+00, 3.18407463e+00, 1.85155956e+00,
   2.27956961e+00, 2.03747479e+00, 2.47744712e+00, 5.20482239e-01,
   3.19212374e+00, 2.68013471e+00, 1.89152655e+00, 3.20586074e+00],
 [ 7.10965649e-01,-5.06620512e-01, 1.48849399e+00, 1.12373448e+00,
   2.93612507e+00, 1.23072049e+00, 1.24548928e+00,-3.95934970e-01,
   9.09932118e-01, 2.29057008e+00, 6.25949842e-01, 4.47153086e-01,
   2.12327784e+00, 5.68072293e-01,-4.39263579e-01, 4.64780007e-01],
 [ 1.28909534e+00, 2.49298452e+00, 2.88330893e+00, 7.76334511e-01,
   3.21107999e+00, 8.97848522e-01, 2.09157963e+00, 9.66974912e-01,
   8.35298720e-01, 7.58564971e-01, 1.00107674e-01, 8.02025358e-01,
  -3.05636317e-01, 3.66982677e+00, 2.80036192e+00, 3.27699695e+00],
 [ 2.01659258e+00, 1.47405086e+00, 3.33813333e+00, 1.97055282e+00,
   6.45208386e-01, 5.89290799e-01, 3.80640070e-01, 2.56460055e+00,
   2.20224202e+00, 6.66615240e-01, 4.14578877e-01,-1.11465402e+00,
   2.41977958e+00, 1.87583477e+00, 4.52943528e-02, 1.81232586e-01],
 [ 1.12621913e+00, 1.59314658e+00,-2.72211736e-02, 9.75571538e-01,
   3.44318752e+00, 8.54387451e-01, 1.91999007e+00,-1.30008347e-01,
   2.39689709e+00, 3.33293156e+00, 2.02893548e+00, 2.13715010e+00,
   2.59195355e+00, 3.55629145e+00, 4.15859757e-01, 1.24073580e+00],
 [ 1.31619789e+00,-8.36809947e-01, 7.31432213e-01, 4.23871617e-01,
   3.02793946e+00, 3.74791582e+00,-6.12843200e-01, 7.94666607e-01,
   5.94248090e-01, 3.13879182e-01, 2.33856632e+00, 1.14160181e-01,
   2.26778403e+00, 3.47633004e+00, 3.00621603e+00, 3.09995032e-01],
 [ 1.93812359e+00,-1.38186896e-01, 2.54192737e+00, 3.14817788e+00,
   2.12914717e+00, 2.81886631e+00, 1.99007471e+00,-2.12798254e-01,
   1.47003545e-01, 1.12216897e+00, 1.87455898e+00, 8.07895432e-01,
   1.07628763e+00, 2.33276559e+00, 1.70326409e+00, 4.61961811e-01]])

In [20]:
IZZI_nd16_loss = 5.465394998351106e-14

IZZI_nd16_alpha = np.array([ 2.35717983, 2.400585  , 0.90157208, 1.86031918, 1.34266081, 1.27207412,
  1.45973957, 0.19202008, 1.12898676,-0.10905987, 1.58448488, 0.26894396,
  1.03048138, 1.65447544, 0.20502352, 1.88140942])

IZZI_nd16_theta = np.array([[ 1.08011044e+00, 1.73551142e+00, 1.69815028e+00, 6.45391953e-01,
   2.39007503e+00, 8.83488233e-01, 3.32636792e+00, 1.37147070e+00,
   1.22765455e+00, 3.10163561e+00, 8.50680784e-01, 2.95741105e+00,
   4.88067243e-01, 1.87908913e+00, 2.23371805e+00,-1.31588939e-02],
 [ 2.43569818e+00, 9.75539080e-01, 1.40215802e+00, 1.35240053e+00,
   3.07094155e+00, 2.24944996e+00, 2.15470875e+00, 6.33790281e-01,
   1.65376819e+00, 5.62864995e-01, 2.76552263e+00, 1.03679756e-01,
   2.11619914e+00, 3.03708519e+00, 1.49842355e+00, 1.33253942e+00],
 [ 1.33039356e+00, 1.17068412e+00, 1.84324760e+00, 2.02011316e+00,
   1.96784238e+00, 9.48276934e-01, 1.61779002e+00, 5.12541068e-01,
   2.35753152e+00, 7.66671615e-01, 1.81998925e+00, 2.49350637e+00,
   1.14465237e+00, 3.32752772e+00, 6.62439426e-01, 1.33098312e+00],
 [ 7.70178481e-01, 2.68391520e+00, 1.19590091e-03,-1.39976145e+00,
   1.56338647e+00, 1.16030729e+00, 1.88954353e+00, 3.80038346e+00,
   2.92006179e+00, 2.11104286e+00, 2.09002778e+00, 1.89551752e+00,
   3.00981818e+00, 3.51548278e+00, 2.04171628e+00, 1.97810006e+00],
 [-1.53166404e+00,-9.81255442e-01, 8.98264933e-01, 7.78173744e-01,
   2.56021366e+00, 1.02688214e+00, 3.80009345e+00, 4.09032410e+00,
   3.91577593e+00, 3.35298617e-01, 1.81974389e+00, 3.42848532e+00,
   1.61929527e+00,-1.63502659e-01, 3.83726802e+00, 6.33246500e-01],
 [ 2.05273099e+00, 3.03675648e+00, 2.71228626e-01, 2.35135825e+00,
   4.67683669e-01, 1.74570914e+00, 1.06776037e+00,-2.14325882e-01,
   2.02772251e+00, 8.58120238e-01, 6.34776938e-01, 1.29426357e+00,
   1.29557779e+00, 4.81304871e-01, 2.01702520e+00,-1.02259569e+00],
 [ 3.08736833e+00, 2.51633850e+00,-1.22749699e-01, 1.05556801e+00,
   3.57209478e+00, 1.20500057e+00, 2.14041620e+00, 6.14895341e-01,
   6.88369723e-01, 1.44136016e+00, 1.75117490e+00, 1.81693875e-01,
   6.78361137e-01,-1.28165661e+00, 5.01084000e-01, 6.63510629e-01],
 [ 8.24226496e-02, 6.43675075e-01,-3.43320871e-01, 3.56185695e+00,
   1.19257480e+00, 2.25410855e-01, 8.18087492e-01, 1.03174274e+00,
   2.59272798e+00, 1.64950224e+00, 2.24697145e+00, 2.36303756e+00,
   3.49270404e+00, 2.38309275e+00, 1.92633646e+00, 3.54983686e+00],
 [ 6.73181838e-01, 4.51709498e+00, 6.75939449e-01, 2.94792789e+00,
  -5.56655246e-01,-6.06839491e-01, 4.02971427e-01, 4.74592076e-01,
   2.53301039e+00, 3.22932300e+00, 2.34478650e+00, 3.22309203e+00,
   1.48562258e+00, 2.19290903e+00,-1.16005794e+00,-3.65080913e-01],
 [ 2.89304988e+00,-3.47175576e-01, 1.78083737e+00, 2.99013692e+00,
   4.01390612e+00, 2.00744474e+00, 3.15771165e+00, 1.31067797e+00,
   2.60340197e+00, 2.22008433e+00, 1.95983153e-03,-2.62055852e-01,
   2.65580443e+00, 6.94410210e-01, 2.41131654e+00, 2.15214899e+00],
 [-7.28264068e-01,-6.05051376e-01, 9.60537048e-02, 2.46634691e+00,
   2.56620067e+00, 1.70088917e+00,-4.66122072e-01, 3.46290577e-01,
   5.32912158e-01, 1.15003566e+00, 1.54801297e+00, 1.72511366e+00,
   3.28389264e+00, 1.75946378e+00, 5.19922459e-01, 1.16592131e+00],
 [ 2.02254730e+00, 1.34352093e+00, 2.41756291e+00, 1.98886526e+00,
   1.90401373e+00, 3.10295987e+00, 1.35232257e+00, 1.85403166e+00,
   4.59592061e+00, 2.99962004e+00, 2.50554359e-01,-5.87747921e-02,
  -5.36067736e-01, 2.16273350e+00, 1.28898854e+00, 1.51906321e+00],
 [ 8.19027278e-01, 2.38323508e+00, 2.17508476e+00, 2.39411860e+00,
  -1.12788040e+00,-1.45515033e-02, 6.56500037e-01, 1.64546450e+00,
   3.03959554e+00, 1.48096272e+00, 7.72537807e-01, 3.10840572e+00,
   1.46924743e+00,-4.22240875e-01, 5.38765158e-02, 1.21574103e+00],
 [ 3.29992012e+00, 1.69517665e+00, 1.49216361e+00, 1.45127629e+00,
   2.03704319e+00,-1.72925131e+00, 2.36031428e+00, 1.90075304e-01,
   1.33377044e+00, 1.12597137e+00, 2.27508971e+00, 1.99232823e+00,
   1.07372641e+00, 1.99804751e-01, 9.11726175e-01, 1.89449031e+00],
 [-9.24472581e-01, 1.16152252e+00,-8.35636345e-01, 2.74519826e+00,
   1.93256342e+00, 3.53832765e+00, 1.29720119e+00, 5.03366992e-01,
   3.18866253e+00, 4.38550228e-01, 9.77733896e-01, 2.60384377e+00,
   1.98286493e+00, 1.64308642e+00, 2.83960465e+00,-4.92140878e-01],
 [ 2.52792532e+00, 6.98899040e-01, 2.51052760e+00, 3.98114155e+00,
   3.05079150e+00, 6.10889207e-01, 1.09532848e-01, 7.67990016e-01,
  -1.64204062e-01, 2.77983607e+00, 5.48550833e-01, 1.41857243e-01,
   3.22274869e+00, 1.20004426e+00, 1.26089844e+00,-4.77877099e-01]])

In [21]:
IZIZ_nd16_loss = 1.7736988068120623e-13

IZIZ_nd16_alpha = np.array([ 2.77937509, 0.14655873, 2.74134791, 2.44634263, 0.5631119 , 3.22894779,
  2.02824802, 0.98169737, 1.80507698, 2.64935934,-0.08760125, 0.58405556,
  1.15297894, 1.00239968, 1.45567505, 1.73049782])

IZIZ_nd16_theta = np.array([[ 1.12089358, 0.88728933, 0.61403631, 0.63238404, 1.45047137,-0.55207591,
   1.0858441 ,-0.7640358 , 3.57009802, 2.00808299, 0.82896931, 3.23712786,
   0.95577558, 3.55980196, 4.14312541, 3.03253552],
 [ 3.92911227, 0.64987274, 0.14740219, 3.92159002, 3.36905927, 2.22935466,
   2.00085198, 3.75719774,-0.2518726 , 0.50262982, 1.22834823,-1.04510251,
   1.06753896, 2.06786337,-0.28427278, 0.57348143],
 [ 2.79770284, 2.77967859, 0.65693513, 0.08851179, 1.49559573, 3.56961785,
  -0.65711263, 1.86430186, 3.15256705, 0.85414423, 2.10556325, 1.01504693,
   0.47472799, 2.01082819, 0.39894911, 2.52691365],
 [ 2.53592427, 0.69599992, 1.91951603, 1.42753816, 1.29352084, 1.22920296,
   2.63373458, 0.69126479, 1.24916446, 2.9996374 , 2.73729015, 0.68848951,
   1.99067646, 2.08268525,-0.11116029, 0.32771935],
 [ 1.90380978,-0.5173576 , 4.16655728, 0.67988376, 2.09050614, 2.46732916,
   2.74503368, 1.18337859,-0.90916208, 2.55698811, 0.22151588, 0.36168678,
   1.66569481, 0.35056091, 0.90811105, 1.02550454],
 [ 3.51788277, 2.18929807, 1.22438931,-0.24651798, 0.14810437, 2.42779884,
   2.21707563, 1.81647858, 1.7941131 , 0.85716293, 3.12170311, 0.40956292,
   2.47782215, 0.75083751, 2.45261107, 0.27830958],
 [ 0.96321983, 3.49038659, 2.04228471, 1.3619095 , 2.68871514, 1.6313675 ,
   0.9751024 , 1.04315092, 0.68583507, 2.4453384 , 3.02554726, 1.72590707,
   3.68042105, 2.32242594,-0.57363729, 2.25884993],
 [ 3.39236951, 1.06970444, 3.56559449, 0.93839359, 1.01552569,-0.00689302,
   1.50913953, 1.28171731, 1.438955  , 3.07060358, 0.15808953, 2.15732455,
   0.47225661, 2.74976944,-0.48048915, 3.17121664],
 [ 1.51091024, 3.15477457, 2.92180169, 1.84949149, 3.02938605, 2.84031523,
  -0.63411341, 1.34195055, 2.05136148, 2.88809546, 0.61071735, 1.73971406,
   2.63185247, 1.14487447,-0.11922796, 2.47256715],
 [ 2.32064447, 2.9231077 , 2.0570048 , 2.25755393, 0.82514067, 0.81165261,
  -0.89131802, 2.58094311, 2.04559837, 4.06639598, 3.01330528, 0.89962566,
   0.80153609, 0.30846653, 1.73921066, 2.24787615],
 [ 1.82708305,-0.21959372, 1.55386434, 0.42194753, 2.40410324, 1.28286813,
   0.85904795, 2.66722854, 2.27283577,-1.20741247, 1.13447402, 1.56248726,
   4.35117647, 2.2446532 , 2.01167805, 2.61954606],
 [ 1.26633331, 3.68388671, 1.83152693, 2.68376148, 1.93820992, 2.98530933,
   3.25264546, 1.02856884,-1.17749124,-0.01872057, 2.37369916, 0.80736406,
   1.42423992, 1.01624582, 1.37891269,-0.60833458],
 [ 0.29194031, 1.19886873, 0.95814602, 0.17317642, 0.72760249, 1.88149172,
   0.65574552, 1.36435733, 1.15642242, 1.2630136 , 2.79904021, 0.63351004,
   0.58487913, 5.03866568,-0.43446505, 1.86493448],
 [ 2.02865228, 1.21316942, 2.49000969, 3.39496399, 0.62712384, 0.14935291,
   1.44916212, 1.7650723 ,-0.68766067, 0.2744943 , 0.51839191, 2.23265074,
   2.7104546 ,-0.53129336, 2.82935126, 1.97760005],
 [ 1.66049469, 4.67803215, 2.03060332, 1.88866041, 1.66064162, 1.65997929,
   0.2350304 , 0.51799097, 0.14728714, 0.89462334, 3.48936634, 0.83548987,
   1.95346033, 2.52535756, 0.86079104, 0.37696314],
 [ 0.70794328, 1.51299057, 2.17639455, 3.76604255, 0.36672761, 2.64131923,
   1.86666564, 2.26625101,-0.7295611 ,-0.37210807, 1.30724333, 3.68464946,
   3.20170831, 1.78271153, 0.50808034, 1.52451733]])

In [22]:
IIZZ_nd16_loss = 2.8934500857354075e-14

IIZZ_nd16_alpha = np.array([1.51109889,2.91790237,2.7571383 ,3.09869356,0.8447634 ,1.29765949,
 3.14368925,0.63333734,2.69461697,2.55890142,1.13590865,2.79754312,
 1.12726064,0.68751306,2.77324293,0.65326248])

IIZZ_nd16_theta = np.array([[ 0.45140248, 0.99188105, 2.04330036, 1.45892797, 1.64196485, 4.26400212,
   2.16101934, 0.66704195, 0.14281982, 1.84334058, 3.16595759, 1.73037505,
   3.25042292, 0.97293736, 2.83569082, 1.1785441 ],
 [ 3.75553794, 0.3373874 ,-0.32693416, 1.02543276, 0.29933861, 3.31119678,
   0.53512426, 0.90213468,-0.68423256, 0.35708974, 2.2320265 , 1.22950408,
   2.37338402, 2.18826979, 1.52382159, 0.35783096],
 [ 1.3587823 , 0.7477793 , 0.67112694, 2.57848186, 2.59881536,-0.85349949,
   2.21776972, 1.07085916, 1.15484913, 0.77385703, 1.67070173, 3.00310212,
   0.68437774, 1.28604126, 2.83162767, 2.60904554],
 [ 1.03750675, 0.82503011,-0.33134634, 0.83546543, 1.16862598, 1.69713405,
   2.12376045,-0.14575859, 0.69973595, 2.58153544, 1.79431859, 1.97926103,
   0.43679917, 3.45087915, 1.78441513, 3.6887802 ],
 [-0.50111262, 2.87562275, 2.15455557, 2.48510479, 1.81722443, 1.72414731,
   2.30518779, 0.28227389,-0.46060601, 3.87214954,-0.98247439, 3.04220161,
   3.07247129, 1.54853833, 2.23478985, 3.44401721],
 [ 1.21678236, 0.79164797, 2.42543021, 1.61223743, 1.85953808, 2.92497292,
   0.97030237, 0.09536459, 0.13894312, 2.04513787, 1.61653603, 4.12635484,
   0.07952874, 2.75761868, 1.87597165, 2.19602406],
 [-0.16582836, 1.84530586, 1.04639648, 2.35979216, 1.580146  , 2.09838467,
   3.04376427, 1.56793722, 0.42158495, 2.12725685, 2.20519   , 1.69184121,
   1.32417465, 1.92270466, 1.02121557, 2.46506559],
 [ 0.93529451, 2.82689072, 1.85675305, 1.31733909,-0.34289979, 2.23704491,
   0.62278161, 1.41185847,-0.31420284, 2.06370879, 0.50506431, 0.67405012,
   3.19244499, 1.47937423, 2.83654604, 1.26494072],
 [ 2.09762124, 0.16316942, 1.29574525, 0.68905598, 2.66859688,-0.64105754,
  -0.39485459, 3.40432067, 1.9767461 , 1.11403009, 1.59117558, 2.86870393,
   0.81472555, 3.45371177, 2.5826858 , 1.9668497 ],
 [ 1.41344227,-0.2311191 , 0.50887754, 3.42389928, 1.29482282, 1.13912677,
   1.41236977, 2.38913375, 1.7062988 , 1.68098655,-0.20841462,-0.0089842 ,
   1.89855575, 2.30398955, 1.7410777 , 2.5211566 ],
 [ 1.54490856, 4.25562323, 0.30059134, 1.74523525, 2.08086125, 1.78809083,
   3.07629851,-0.8958667 , 2.23907562, 0.15238858, 1.67780903, 1.82985732,
   3.44573914, 2.99299155, 0.94268959, 0.82127763],
 [ 1.3480374 , 2.67968505, 1.15191538, 1.37495509, 1.19106987, 2.24613698,
   1.53210821, 0.76530014, 0.15321663, 2.46210229, 0.58859371, 1.31862294,
  -0.83114827, 0.04995639, 1.28794929, 3.16865855],
 [ 1.97332185, 0.28856194, 2.02779561, 2.25175244, 2.25898843, 1.34827227,
   0.39070213, 2.32542757, 0.36795472,-0.41679768, 2.11931103, 1.23095056,
   2.53302702, 1.38865744, 2.39713018,-0.1393088 ],
 [ 2.42347   , 1.01620461, 1.48165808, 1.78870996,-0.35970522, 0.19638484,
   1.51844472, 1.54126358, 1.56163691, 2.48154802, 0.75516398, 4.23151919,
   0.40454903,-0.24842336, 0.45654744, 0.15147015],
 [ 2.34638887, 2.17689838, 1.90478947, 0.39662988, 4.40325388, 3.44625485,
   1.21127364, 1.38854207, 0.52197572, 0.96595575, 0.93574861, 0.18193476,
   0.44193874, 2.10813203, 2.87363583, 0.55848482],
 [ 1.07075978,-0.09453698, 2.63883448, 0.84982558, 2.74327764, 1.16808484,
   2.24349237, 1.04942916, 1.0497671 , 1.09572903, 1.83558644, 1.08550779,
   2.26642708, 1.74369305, 0.67632247, 2.39128412]])

In [23]:
XYYX_nd16_loss = 5.4396147186001625e-14

XYYX_nd16_alpha = np.array([1.41350815,0.99569326,2.13797166,0.74532049,1.35026344,1.21335256,
 0.49672062,2.06386493,1.05026635,0.3133549 ,0.63799669,0.39043071,
 1.0629971 ,2.14497379,2.09335355,1.08437802])

XYYX_nd16_theta = np.array([[ 3.10576567, 1.64269305, 0.92698283, 3.0996815 , 2.96436942, 0.78323386,
   2.91578467, 2.44490488, 0.26662465, 1.26282477, 2.79633139, 0.78826354,
   2.89666183, 2.77965474, 1.23441004,-0.38471184],
 [ 1.55403755, 0.83723551, 2.56653639, 1.22810109, 1.60824618, 0.95811278,
   1.31384422, 2.70121952, 2.15300757, 2.29266464, 4.86254932,-0.07085847,
  -1.00911898, 2.77575171, 0.48085124, 1.54194264],
 [ 2.66269208, 2.78083751,-0.3403298 , 3.2034721 , 4.11101278, 1.8190515 ,
   1.15772717, 2.05468693, 0.53108734, 3.17025367, 1.31087776, 0.96151312,
   2.00595309, 0.82891133, 1.4939096 , 2.34330914],
 [ 1.246514  , 0.27978821, 0.41135493, 0.96378431, 0.68709136, 1.30494843,
  -0.33103167, 1.36267679, 2.99908238, 1.01789708, 1.33746939, 1.82933612,
   2.39446036, 0.27068218, 4.08704852, 4.07189993],
 [ 1.71623448,-0.4243245 , 1.80172682, 1.41203616, 0.14441764, 0.25805851,
   0.90093912, 1.36974667, 2.00308362, 0.2850913 ,-0.68724945, 2.50758274,
  -0.27410604, 2.49391856, 0.79426592, 2.12924192],
 [ 2.16085347, 2.04748992, 1.29344791, 1.61069111,-0.31565393, 1.47298867,
   0.64391187, 0.29601372, 2.29283349, 0.54977406, 3.38153922, 1.41059629,
   0.98661191, 3.88795741, 2.98898116, 0.85522354],
 [ 1.59739626, 0.96540726, 1.76231841, 1.12249334, 1.69121629, 3.41499565,
   2.28881445, 0.10578263, 2.781183  , 0.92217353, 1.22898999, 1.02619983,
   2.31629871, 0.6735605 , 0.86224619, 0.67109879],
 [ 3.35441064, 2.04628878, 0.78914246, 1.37309599, 0.75457256, 2.1702477 ,
   1.11208534, 1.99352161, 2.80102938, 1.79621876, 2.076533  , 2.0002831 ,
   0.92976519, 0.26361951, 3.70820731, 0.84556593],
 [-0.32032392, 3.33393337, 1.04000601, 1.6802887 , 2.20006617, 1.4434136 ,
   0.22583022, 3.32811923, 1.85485444, 3.79503906,-1.0982232 , 1.68952151,
   1.71081155, 0.83691187, 1.44204188, 4.0309842 ],
 [ 4.63206819, 2.37197347,-0.25364075, 1.53186842, 2.65899669,-0.44911468,
   2.45883958, 1.19373584, 1.92024946, 1.9410246 , 2.3309026 , 0.18355279,
   2.11150766, 2.89544052, 0.10986671,-0.30313928],
 [ 1.46532731, 0.22086712, 2.24945337, 3.04221284, 2.73150371, 1.10417471,
  -0.29149829, 0.40738149, 0.42071579, 2.16184028, 0.76510995,-0.51022015,
   1.28815192, 1.22848921, 1.59197041, 2.84053512],
 [ 1.38560428, 2.48038162, 0.64311735,-0.33815716, 1.72227248, 1.14674615,
   1.58073696, 1.36176755, 2.97199465, 3.39955508, 1.26886524, 0.31620232,
   0.70735911, 1.27485046, 0.54304976, 1.05102957],
 [-0.5000222 , 1.4382954 , 2.48565929, 2.18015269, 0.34347082, 1.03914671,
   0.64034257, 2.34170107, 1.5551461 , 1.63388225, 1.25220496, 2.03301283,
   2.11050897, 3.86082621, 1.36164569, 1.19898065],
 [-0.42412076, 0.75684669, 1.66309884, 2.00999108, 3.79018527, 2.44381227,
  -0.43576859, 0.7454744 , 3.42627794, 3.01758865, 2.1394801 , 2.38629053,
   2.56371095, 2.48034574, 3.89314019, 2.38987242],
 [-0.05478765, 0.6701026 , 3.38451876, 1.32370621, 2.40921197, 1.82568541,
   1.93924728, 2.08809522, 1.33020136, 3.43488592, 2.44973856, 0.1754229 ,
  -0.22471464, 2.81722868, 1.18937   , 0.74339034],
 [-0.09466708, 2.14132914, 2.87602475, 0.40421294, 1.89886774, 0.73972368,
   2.34944233, 2.45604821, 1.07389485, 3.51872608, 3.02028054, 3.51752965,
   3.13415477, 4.27064683, 1.7110042 , 0.80748118]])

In [24]:
YXXY_nd16_loss = 1.0096707992927935e-13

YXXY_nd16_alpha = np.array([0.50039615,1.57252366,1.13445459,0.59398413,1.57426647,1.95621593,
 1.09701775,0.18990201,1.63758855,1.67059905,0.51358039,1.70416524,
 2.77453008,1.7326699 ,2.57397469,1.42272066])

YXXY_nd16_theta = np.array([[ 1.02006322, 3.21169887, 1.4944074 , 1.9424202 , 0.69488628, 0.30557893,
   2.38622769, 2.64604709, 0.3777854 , 0.17892445, 0.28451124, 1.49107768,
   1.66877292, 1.92599357, 1.77538354, 1.8097415 ],
 [ 0.69360603, 1.35946907, 1.17799038,-0.25628953, 1.30022054, 2.18638793,
   0.29013522, 3.0854195 , 3.43339078, 2.7677606 , 4.62156585, 1.67925393,
   0.36689234, 1.96775777, 0.84910598, 1.50665269],
 [ 3.05192062, 0.49555644, 0.67557883, 3.59137466, 1.80921427, 0.59282617,
   1.55807978, 2.26890739,-0.59680761, 0.92665401, 2.57326325, 0.63098956,
   0.38789983, 0.09727572, 2.66477802, 1.82388911],
 [ 2.78641118,-0.91123023, 2.42374359, 2.96051037, 2.65882248, 2.84786629,
   1.04803146, 2.14102036, 1.81731545, 3.97889044, 1.12144323, 2.38212445,
   0.73968494, 1.75031631, 1.36731801, 0.86527791],
 [ 1.10837826, 2.28547995, 2.35643479, 2.08093683,-0.54583321, 1.16246443,
   1.61695666, 3.43762999, 0.30644748, 1.88658469, 1.2451027 ,-0.4374903 ,
   2.90805763, 2.95231085, 3.89994539, 0.78466653],
 [ 3.71990288, 0.47126014, 0.62343805, 1.30263128, 0.71141753, 1.69463169,
   0.70604374, 0.54574608, 1.83029891, 2.71454556, 1.09223067,-0.10422528,
   1.50047805, 2.84724175, 2.07538991, 4.53496056],
 [-0.77181784, 1.48886786, 3.25699675, 2.39148404, 0.41246631, 2.86125268,
   2.29177958, 1.52673597, 1.42904643, 2.09859389, 0.31183534, 1.92150123,
   1.09613894, 1.65407005, 1.81520546, 3.29500404],
 [ 0.59293946, 3.70826354, 0.44382505, 3.37927631, 1.39550846, 1.30644647,
   0.85468998, 0.03387126, 2.47248054, 1.12832777, 2.80289822, 1.56212247,
  -0.11005796, 2.66860598, 1.31570049, 2.04084131],
 [ 2.14588101, 2.86288779, 1.3617731 , 1.81030125, 1.04622254, 0.12421233,
   2.3562922 , 3.12158152, 1.83333542, 1.96783042,-0.64504606, 2.25889537,
   1.92040603, 1.77911701, 0.08033118, 1.13502894],
 [ 0.84148894, 1.43414425, 0.50070269, 0.73629906, 0.33597531, 3.11851533,
   1.17647383, 0.78275529,-0.17534136, 1.2971095 , 4.48827893, 1.39466196,
   2.59533366, 1.13690092, 2.0865314 , 4.02589092],
 [ 0.68631745, 2.47841506, 2.42808897, 2.47287198, 3.54912733, 3.3606897 ,
   2.78589547, 0.22061166, 1.82651943, 3.37568932, 1.5344684 , 0.12256324,
   0.84674037,-1.24867402, 0.88109923,-0.09700522],
 [ 0.08631663, 0.96830486, 2.84924101, 3.19132256, 0.65435076, 1.75989687,
   3.20184268, 2.47771064, 0.38027156, 1.08988001, 0.44104702,-0.9958397 ,
   1.29836823, 1.48298172, 1.64961623, 0.50431318],
 [ 2.03367176, 0.39955833, 0.93004446, 0.7389398 , 1.22448976, 1.15655749,
   0.79241284,-1.15906571, 1.45115626, 1.19954029, 0.2985142 , 1.49415756,
   3.59351823,-0.32536684, 1.31323367, 1.25208175],
 [ 1.56738859, 2.56467843,-0.41829497, 1.83386841, 0.93144614, 1.32335793,
   1.50544521, 1.8773791 , 1.29736761, 1.59850656, 0.36055106, 1.57184753,
   1.49874827, 3.10571921, 1.45460608, 0.14802231],
 [ 2.16770996, 1.10462852, 0.82239342, 2.83464928, 2.32767107, 1.71672512,
   2.53110021,-0.5791255 , 1.11249086, 2.62699059, 1.1096241 , 0.529587  ,
   2.08040006, 2.82470047, 2.38489634, 3.27128923],
 [ 1.27783132, 2.72470105, 0.73677777, 1.00513971, 3.0625897 , 2.50727842,
   2.89283492, 2.0081514 , 1.76124874, 0.86091252, 1.36218759, 0.1712776 ,
   3.24286783, 0.8183618 , 1.68660882, 2.5667154 ]])

In [25]:
XXYY_nd16_loss = 1.0620082707311773e-13

XXYY_nd16_alpha = np.array([1.4374415 ,1.01739782,2.26915995,1.11969346,2.60568931,1.11431704,
 0.81969038,1.86981594,2.05898402,2.04414789,0.46102625,0.40333794,
 2.34227156,0.93513822,2.39007701,0.66351455])

XXYY_nd16_theta = np.array([[ 2.39570413e+00, 1.50519661e-02, 6.29655208e-01, 1.16897236e+00,
   1.41695275e+00, 3.10610705e+00, 2.92551187e+00, 3.99381446e+00,
   2.48211085e+00, 4.79681091e-01, 7.21064560e-01,-9.82450821e-01,
   2.70922740e+00, 1.04647782e+00, 1.09448114e+00, 2.51408335e+00],
 [ 2.00833408e+00, 2.75143900e+00,-4.76174996e-01, 7.85365721e-01,
   1.28775575e+00, 1.32802153e-01, 3.04887350e+00, 8.81585607e-01,
   2.76227440e-01, 1.06775399e+00, 2.17486490e+00, 2.66081485e+00,
   1.81411374e+00, 2.16048820e+00, 2.05657866e+00, 3.67676103e+00],
 [ 1.78499248e+00, 2.31504141e+00,-2.09871672e-01, 1.01717273e-01,
   4.47274630e-01, 1.15581543e+00, 2.74928674e+00, 3.58951545e+00,
   2.16102964e+00, 1.26137198e+00, 1.16297157e+00, 8.53946348e-01,
   1.74435894e+00, 8.91946065e-01, 2.26680634e+00, 2.46559614e+00],
 [ 1.64728175e+00,-5.79078305e-01, 2.18887458e+00, 6.94117223e-01,
   6.81545640e-01, 3.74547460e+00,-5.40626809e-02,-6.11623327e-01,
   1.09517302e+00, 5.29590060e-01, 3.56145748e+00, 1.22430371e-01,
   7.15750747e-01, 3.00118291e+00, 2.20929799e+00, 2.92065187e+00],
 [ 1.03000074e+00,-1.33198084e-01, 1.68813676e+00, 7.58350275e-01,
   1.44869389e+00, 3.54460265e+00, 2.64160600e-01,-5.85041502e-01,
   2.91047560e+00, 2.65839053e+00, 1.76143257e+00, 1.68083739e+00,
  -2.68744191e-01, 4.62179928e-01, 3.59268136e+00, 2.41980778e+00],
 [ 3.40970399e+00, 1.65059913e+00, 3.05984189e-01, 1.01018739e+00,
   2.37417586e+00, 2.26095540e+00, 6.28085454e-02, 2.33211007e+00,
   1.27675940e-01, 2.84640434e+00, 1.70929174e+00, 2.06278843e+00,
   2.79406812e+00,-3.38071284e-01, 1.16274581e+00,-1.79289117e-01],
 [-2.83404503e-01, 7.62608264e-01, 1.39447972e+00, 2.15232678e-01,
   3.20120277e+00, 1.22273409e+00, 2.07402724e+00, 8.53916857e-01,
   2.35734469e+00, 2.89404176e+00, 1.42658774e+00, 2.70425082e+00,
   2.53340498e+00, 2.13164436e+00, 2.49239918e+00, 5.85519147e-01],
 [ 1.20146497e+00, 1.14820005e+00, 2.27629246e-01, 9.21606192e-01,
   3.50393657e+00, 3.64663375e+00, 1.96400844e+00, 1.70960280e+00,
   6.18599423e-01, 5.11037358e-01, 9.82404672e-01, 1.97546282e+00,
   2.83632089e+00,-5.68500559e-03, 2.75126334e+00, 1.81995013e+00],
 [ 2.51723905e+00, 3.82131342e+00, 2.04393766e+00,-4.76222850e-05,
   2.88930285e+00,-8.75625580e-01, 2.81385815e+00, 8.56556611e-01,
   1.05516083e+00, 2.44373301e+00,-2.31517885e-01, 3.36430223e-01,
   2.20373182e-01, 3.32533517e+00, 1.59518523e+00, 1.92667451e+00],
 [ 6.73351236e-01, 1.67610810e+00, 8.42003309e-01, 7.58328818e-01,
   4.08085364e+00, 2.08706739e+00, 4.22741774e-01, 9.78561669e-01,
   1.98493558e+00, 1.69338995e+00, 1.32441209e+00, 6.92737562e-01,
   1.30912437e+00, 2.08727669e+00,-5.09840376e-01, 1.00899245e+00],
 [ 1.31174791e+00, 3.64820239e+00, 2.90077856e+00, 3.16161799e+00,
   1.31844493e+00, 9.44172005e-01,-1.08038743e+00, 1.80339140e+00,
  -7.06984054e-01, 1.74311472e+00, 1.03146090e-01, 2.94938371e+00,
   1.10730415e+00, 1.69177659e+00,-1.22585744e-01, 2.06636084e+00],
 [ 2.57534124e+00,-1.10888302e+00, 2.34962247e+00,-1.14661548e+00,
   1.03590138e+00, 3.52199050e-01, 4.04817829e-01, 1.52471369e+00,
   3.20248996e+00, 2.60253387e+00, 2.05372430e+00, 2.42417853e+00,
   2.06316387e+00, 3.26709336e+00, 2.11008212e+00, 2.77842274e+00],
 [ 3.24197329e+00, 1.64148361e+00, 2.67035899e+00, 2.53880144e+00,
   3.41381464e-01, 9.76601606e-01, 1.20967997e+00, 1.67339318e+00,
  -3.65279855e-01, 2.68847278e+00, 1.39955854e+00, 2.55578014e+00,
   9.83161224e-01, 4.93013407e-01,-1.77051635e-02, 5.66012590e-01],
 [ 2.12959080e+00, 1.81022097e-02, 8.21681932e-01,-2.46119823e-01,
   1.86662953e+00, 2.04539529e+00, 1.99122155e+00,-7.42578270e-01,
   1.95126465e+00, 1.00802292e+00, 9.64658991e-01, 1.51652094e+00,
   1.37661608e+00, 1.76869145e+00, 3.64695108e+00, 3.75577743e+00],
 [ 1.33796394e+00, 5.99225590e-01, 1.19972630e+00, 2.16809527e-01,
   1.59767120e+00, 1.83964716e+00, 3.07622583e+00, 1.78047745e+00,
   2.54088478e+00, 2.07112207e+00, 4.96724851e-01, 1.66769296e+00,
   1.89527865e+00, 6.41489751e-04, 2.20124602e+00, 3.27392697e+00],
 [-1.66887838e-01, 1.77154082e+00, 2.56399335e+00, 1.33605284e+00,
   1.27179178e+00, 3.23024086e-01, 3.13139084e+00, 1.17432952e+00,
   1.70543743e+00, 2.02221671e+00, 1.57210298e+00, 2.16947453e+00,
   1.46683426e+00, 2.18149130e+00, 6.23290792e-01, 1.13358193e+00]])

In [26]:
YYXX_nd16_loss = 6.051977802793299e-14

YYXX_nd16_alpha = np.array([ 1.82985639, 0.95401642, 1.90269283,-0.07621668, 1.54353149, 1.76898623,
  1.36829511, 2.14565411, 2.81397505, 0.15128809, 2.48931217, 1.62220061,
  1.44112618, 2.29917893, 1.31296993, 2.13192204])

YYXX_nd16_theta = np.array([[-0.38984436, 3.23639731, 3.45912825, 2.36778028, 1.32971249, 2.02488327,
   0.09117307, 2.2487493 , 1.10939547, 2.56069871,-0.07245493, 3.7083275 ,
   2.30411067, 0.85781705, 1.53447359, 1.04263239],
 [-0.39374778, 1.19850852, 1.9402063 , 0.76971162, 0.6609587 , 2.09671399,
   1.42624825, 0.53497966, 2.74135434, 0.66649038, 0.7885915 , 0.27143026,
   2.91092247,-0.58210494, 0.11542036, 2.37540071],
 [ 4.15586446, 1.36091125, 0.10571558, 0.6295235 ,-0.30604884, 0.5659527 ,
  -0.03998186,-3.12117244, 1.38244716, 3.55024443, 2.18814274, 2.42354434,
   4.7328422 , 3.30171526, 1.466077  , 1.11789277],
 [ 0.69593823, 1.20188681, 2.8516411 , 3.94713979, 3.13754123, 2.11797489,
   0.18688551, 3.66028167, 2.04585505,-0.04970315, 3.29288317, 0.06706737,
   1.47261243, 0.99507324, 3.19977989,-0.37476564],
 [ 2.06431383, 2.32072256, 2.20597599, 1.28483724, 2.03808145, 0.00969706,
   1.07978202, 4.03903701, 0.79271504, 2.69493748, 2.44968996, 1.07133882,
   1.29913843, 0.81346631, 0.93619944, 3.62027076],
 [ 2.07144099, 1.88500974,-0.36797874, 0.37516474, 2.02360314,-0.53127908,
   1.95010579, 3.09626715, 3.70820228, 0.57044675, 0.98809686, 2.22554763,
   0.64856681, 0.52933412, 1.8420254 ,-0.17114251],
 [ 0.34723141, 0.85831219, 2.25375554, 0.8653218 , 1.26783892, 2.56709757,
   2.58995314, 0.681437  , 1.8574668 , 1.35942892, 0.63917514, 0.0696373 ,
   0.51052298, 1.21756589, 1.25397356, 1.91016929],
 [-0.46558528, 3.30703232,-0.29983813, 1.6229116 , 2.34975308, 3.04923077,
   0.96482994, 2.09070113, 1.36801058, 2.89474809, 0.38164972, 4.61202845,
   2.75183399,-0.01248454, 1.65713815, 2.9779645 ],
 [ 0.27170548, 3.2044834 , 0.93160799, 1.41240661, 2.09402367, 2.24041425,
   2.39243148, 3.02522341,-0.31178689,-0.53491285, 1.36010884, 1.52604229,
   3.36482626, 1.26522777, 1.94258717, 1.48822233],
 [ 2.16952183,-0.29217143, 0.52017048,-0.24457574, 1.00418057, 3.00625526,
   0.98634209, 2.21554896, 3.03392953, 1.38417497, 2.61834936, 1.19819239,
   0.98351348, 1.22644734, 1.00816634, 2.49555737],
 [ 3.27162221, 2.74305919, 2.88842026, 0.33677999, 0.95944983, 1.78578103,
   1.08115544, 3.05940525, 1.38874324, 0.07094338, 1.24751969, 1.28312364,
   1.94591931, 0.01802341,-1.70220557,-0.08247808],
 [ 0.24279671,-0.81785828, 3.61523046, 1.60402499, 2.04249566, 1.28801567,
   2.6031133 , 2.36199366, 1.76734172,-0.40096969, 1.76045232, 0.44138396,
   3.07727129, 2.61631059, 2.00151731, 0.53152129],
 [ 1.54207787, 2.7550851 , 1.528297  ,-0.04737548, 1.26192255, 1.51484167,
   2.01597525, 0.18550092, 0.62350701, 1.06592065, 0.90432646, 1.23653608,
   0.73387271, 1.23824878, 3.45629963, 1.68043277],
 [ 2.64731856, 1.98160805, 0.76172446, 2.00582247, 0.14575144,-0.44869159,
   2.73465475,-0.06272034, 1.23958204, 0.99900506, 1.93424692, 1.5373627 ,
   1.87134697, 3.92653744, 1.47282415, 0.91587488],
 [ 2.2877366 , 2.5448317 , 2.76305887, 1.89455839, 0.4552489 , 0.54012328,
   2.68248957, 0.70068361, 1.1717156 , 0.55527483, 0.71847066, 2.99401457,
   3.45093352, 0.44297477, 0.73921205, 0.8604468 ],
 [ 1.33929146, 2.7029682 , 2.89357714, 1.62789229, 1.04914843, 1.10892626,
   3.16106215, 0.13570184, 0.7025735 ,-0.74629986, 2.31610254, 0.90369571,
  -0.34045186, 2.29483595, 1.78017333, 2.21124613]])

### Coefficients

In [27]:
def h2mol_data(hhdis):
    """
    Molecular data for H2 molecule in a minimal basis.

    Argument:
    hhdis -- bond distance in Angstrom
    """

    # Parameters
    basis = "sto-3g"
    multiplicity = 1
    charge = 0

    # XYZ coordinates
    geometry = [("H", (0,0,0)), ("H", (0,0,hhdis))]

    # Define molecule
    molecule = MolecularData(geometry, basis, multiplicity, charge)

    # Run HF calculation
    hf_molecule = run_pyscf(molecule, run_scf=1)

    # HF energy
    ehf = hf_molecule.hf_energy

    # Nuclear repulsion scalar
    hnuc = hf_molecule.nuclear_repulsion

    # Electron intergals
    h1, h2 = get_tensors_from_integrals(hf_molecule.one_body_integrals, \
                                        hf_molecule.two_body_integrals)
    v2 = h2 * 2

    return hnuc, h1, v2

In [28]:
def get_ham_coeffs(hhdis):
    """
    Hamiltonian coefficients in the JWT.

    Argument:
    hhdis -- bond distance in Angstrom
    """
    # Initialize
    gvec = np.zeros((8))
    # Paramters
    hnuc, h1, v2 = h2mol_data(hhdis)

    # Coefficients
    gvec[0] = hnuc
    gvec[0] += ( h1[0, 0] + h1[1, 1] + h1[2, 2] + h1[3, 3] ) / 2
    gvec[0] += ( v2[0, 1, 1, 0] + v2[2, 3, 3, 2] + v2[0, 3, 3, 0] + v2[1, 2, 2, 1] ) / 4
    gvec[0] += ( v2[0, 2, 2, 0] - v2[0, 2, 0, 2] + v2[1, 3, 3, 1] - v2[1, 3, 1, 3] ) / 4

    gvec[1] = - h1[0, 0] / 2
    gvec[1] -= ( v2[0, 1, 1, 0] + v2[0, 3, 3, 0] + v2[0, 2, 2, 0] - v2[0, 2, 0, 2] ) / 4

    gvec[2] = - h1[2, 2] / 2
    gvec[2] -= ( v2[2, 3, 3, 2] + v2[1, 2, 2, 1] + v2[0, 2, 2, 0] - v2[0, 2, 0, 2] ) / 4

    gvec[3] = v2[0, 1, 1, 0] / 4

    gvec[4] = ( v2[0, 2, 2, 0] - v2[0, 2, 0, 2] ) / 4

    gvec[5] = v2[0, 3, 3, 0] / 4

    gvec[6] = v2[2, 3, 3, 2] / 4

    gvec[7] = v2[0, 3, 1, 2] / 4

    return gvec

## Expectation value

In [29]:
def trace_out_qumode(state, nfock):
    """
    Performs a partial trace over the qumode part of a density matrix,
    leaving the reduced density matrix for the qubit.

    Arguments:
    state -- statevector for qubit-qumode system
    nfock -- Fock cutoff
    """
    # Initialize
    if not isinstance(state, qt.Qobj):
        qstate = qt.Qobj(state)
    else:
        qstate = state

    # Dimension matching
    dims = [[2, nfock], [1, 1]]
    tstate = qt.Qobj(qstate.data, dims=dims)

    # Get density matrix
    rho_qubit = tstate.ptrace(0)

    return rho_qubit

In [30]:
def include_hadamard(U):
    """
    Hadamard test involving a QuTip operator.
    """
    # Initialize
    dim = U.full().shape[0]

    # Build
    op1 = qt.tensor(hadamard(), qt.qeye(dim))
    op2 = qt.tensor(qt.basis(2, 0).proj(), qt.qeye(dim))
    op2 += qt.tensor(qt.basis(2, 1).proj(), U)

    return op1 * op2 * op1

In [31]:
def psi_uni_psi_hadamard(state, U):
    """
    Expectation value.

    Arguments:
    state-- qumode state in QuTip
    U -- qumode unitary
    """
    # Operators
    nfock = U.shape[0]
    U = qt.Qobj(U)

    # |0, state>
    cstate = qt.tensor(qt.basis(2, 0), state)

    # Hadamard-included operator
    T = include_hadamard(U)

    # T |0, state>
    psi = T * cstate

    # Extract qubit RDM
    qubit_rho = trace_out_qumode(psi, nfock)

    # <Z>
    ov = qt.expect(qt.sigmaz(), qubit_rho)

    return np.real(ov)

In [32]:
def energy_val(Xvec, gvec, nfock):
    """
    Energy function.

    Arguments:
    Xvec -- ansatz parameters
    gvec -- Hamiltonian coefficients
    nfock -- Fock cutoff for the qumode
    """
    # Parameters
    alphavec, thetamat = unpack_params_snap_disp(Xvec, nfock)

    # Unitary
    U = snap_disp_ansatz(alphavec, thetamat, nfock)

    # |psi> = U |0>
    vac = qt.basis(nfock, 0).full()
    state = qt.Qobj(np.matmul(U, vac))

    # Unitaries
    U_ZIII = snap_disp_ansatz(ZIII_nd16_alpha, ZIII_nd16_theta, nfock)
    U_IZII = snap_disp_ansatz(IZII_nd16_alpha, IZII_nd16_theta, nfock)
    U_IIZI = snap_disp_ansatz(IIZI_nd16_alpha, IIZI_nd16_theta, nfock)
    U_IIIZ = snap_disp_ansatz(IIIZ_nd16_alpha, IIIZ_nd16_theta, nfock)
    U_ZZII = snap_disp_ansatz(ZZII_nd16_alpha, ZZII_nd16_theta, nfock)
    U_ZIZI = snap_disp_ansatz(ZIZI_nd16_alpha, ZIZI_nd16_theta, nfock)
    U_ZIIZ = snap_disp_ansatz(ZIIZ_nd16_alpha, ZIIZ_nd16_theta, nfock)
    U_IZZI = snap_disp_ansatz(IZZI_nd16_alpha, IZZI_nd16_theta, nfock)
    U_IZIZ = snap_disp_ansatz(IZIZ_nd16_alpha, IZIZ_nd16_theta, nfock)
    U_IIZZ = snap_disp_ansatz(IIZZ_nd16_alpha, IIZZ_nd16_theta, nfock)
    U_XYYX = snap_disp_ansatz(XYYX_nd16_alpha, XYYX_nd16_theta, nfock)
    U_YXXY = snap_disp_ansatz(YXXY_nd16_alpha, YXXY_nd16_theta, nfock)
    U_XXYY = snap_disp_ansatz(XXYY_nd16_alpha, XXYY_nd16_theta, nfock)
    U_YYXX = snap_disp_ansatz(YYXX_nd16_alpha, YYXX_nd16_theta, nfock)

    # Energy
    en = gvec[0].copy()

    en += ( gvec[1] * psi_uni_psi_hadamard(state, U_ZIII) )
    en += ( gvec[1] * psi_uni_psi_hadamard(state, U_IZII) )

    en += ( gvec[2] * psi_uni_psi_hadamard(state, U_IIZI) )
    en += ( gvec[2] * psi_uni_psi_hadamard(state, U_IIIZ) )

    en += ( gvec[3] * psi_uni_psi_hadamard(state, U_ZZII) )

    en += ( gvec[4] * psi_uni_psi_hadamard(state, U_ZIZI) )
    en += ( gvec[4] * psi_uni_psi_hadamard(state, U_IZIZ) )

    en += ( gvec[5] * psi_uni_psi_hadamard(state, U_ZIIZ) )
    en += ( gvec[5] * psi_uni_psi_hadamard(state, U_IZZI) )

    en += ( gvec[6] * psi_uni_psi_hadamard(state, U_IIZZ) )

    en += ( gvec[7] * psi_uni_psi_hadamard(state, U_XYYX) )
    en += ( gvec[7] * psi_uni_psi_hadamard(state, U_YXXY) )
    en -= ( gvec[7] * psi_uni_psi_hadamard(state, U_XXYY) )
    en -= ( gvec[7] * psi_uni_psi_hadamard(state, U_YYXX) )

    return en

## VQE

In [33]:
def opt_vqe_snap(hhdis, ndepth, maxiter=100, method='COBYLA', verb=0, \
                 threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    hhdis -- H-H bond distance in Angstrom
    ndepth -- ansatz circuit depth
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- print additional things or not
    threshold -- error tolerance
    Xvec -- optional initial guesses
    """
    # Hamiltonian parameters
    nfock = 16
    gvec = get_ham_coeffs(hhdis)

    # Bound parameters
    alpha_min = 0.0
    alpha_max = 10.0
    theta_min = 0.0
    theta_max = np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([alpha_min, alpha_max])
    for _ in range(ndepth * nfock):
        bounds.append([theta_min, theta_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        alpha = np.random.uniform(0, 3, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        Xvec = pack_params_snap_disp(alpha, theta)

    # Loss function
    obj_fun = partial(energy_val, gvec=gvec, nfock=nfock)

    # Intermediate values
    iteration_step = 0
    print_freq = 10
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {obj_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    return result.fun, result.x

## Explore

### Imported data

For H-H bond distances = $ (0.25, 0.35, \cdots, 3.25) $.

In [34]:
dis_angstrom = np.array([0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25,
       1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95, 2.05, 2.15, 2.25, 2.35,
       2.45, 2.55, 2.65, 2.75, 2.85, 2.95, 3.05, 3.15, 3.25])

fci_en = np.array([-0.3122699 , -0.78926939, -0.9984156 , -1.09262991, -1.12990478,
       -1.13711707, -1.12836188, -1.11133942, -1.09034218, -1.06792966,
       -1.04578314, -1.02505436, -1.00648693, -0.99047634, -0.97712962,
       -0.96633454, -0.95783297, -0.95128976, -0.94634974, -0.94267779,
       -0.93998171, -0.93802086, -0.93660526, -0.93558937, -0.93486413,
       -0.93434899, -0.93398498, -0.93372922, -0.9335506 , -0.93342668,
       -0.93334128])

### Tests

In [ ]:
nind = 0
ndepth = 2

en, Xvec = opt_vqe_snap(dis_angstrom[nind], ndepth, maxiter=10, method='BFGS', \
                        verb=1, threshold=1e-08)
en